# CreatorsHive Sコース昇格課題　ー機械学習編ー  
## 課題  
Titanicデータセットから機械学習を利用して，顧客が生存したかを推定しなさい．  
処理の流れは指定されたとおりに行いその実行コードと実行結果をJupyter-Notebookで保存して提出しなさい．  
## データセット内容  
今回利用するデータセットはTitanic号に載っていた顧客の情報とその顧客が事故後に生存したかが記載されているデータセットである．  
データのカラムは以下の通りである． 
* PassengerId：顧客ID  
* Survived：生存したかどうか
* Pclass:階級
* Name：氏名
* Sex：性別
* Age：年齢
* SibSp：一緒に乗船した兄弟・配偶者の人数
* Parch：一緒に乗船した親・子供の人数
* Ticket：チケット番号
* Fare：乗船料金
* Cabin：部屋番号
* Embarked：乗船した港（C = Cherbourg, Q = Queenstown, S = Southamton)
## 使用ライブラリ
* numpy
* pandas
* scikit-learn

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame,Series
import sklearn

## 1.1 データ読み込み  
pandas で train.csv を読み込み、読み込んだデータの先頭 5 行を表示しなさい．

In [2]:
df=pd.read_csv('train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df.shape

(891, 12)

## 1.2 前処理：不要なカラムの削除
読み込んだデータのカラムで前処理が難しいもの，もしくは生存者を推定する際に必要でないものを考え，削除しなさい．  
取り除いたデータの先頭5行を出力しなさい．  
例：Cabin（部屋番号）は特に規則性がなくまた、前処理が難しいため削除する．  

ラベル：Survived  
いらない・前処理が難しい特徴量：Ticket,Cabin,Fare,Name

In [4]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
drop_col=['Ticket','Cabin','Fare','Name','PassengerId']
df=df.drop(drop_col,axis=1)

## 1.3 前処理:カテゴリデータの加工
カラムの中には数値でないデータが存在する．  
そのデータを機械学習で利用できるように数値に変換をする．  

### 1.3.1 ラベルエンコーディング 
値が２種類のカラムを探しそのカラムの値を0,1に変換しなさい．  
処理後のデータの先頭5行を出力しなさい．  

2値のカラム：Sex

In [6]:
sex_mapping={'male':0,'female':1}

In [7]:
df['Sex']=df['Sex'].map(sex_mapping)

In [8]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked
0,0,3,0,22.0,1,0,S
1,1,1,1,38.0,1,0,C
2,1,3,1,26.0,0,0,S
3,1,1,1,35.0,1,0,S
4,0,3,0,35.0,0,0,S


### 1.3.2 one-hotエンコーディング  
値が3種類以上ある場合、ラベルエンコーディングを適応させることはできない．  
one-hotエンコーディング，もしくはダミー変数化を利用して数値に変換しなさい．  
処理後のデータの先頭５行を出力しなさい．  

In [9]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Embarked      2
dtype: int64

In [10]:
df=df.dropna(subset=['Embarked'])

In [11]:
df=pd.get_dummies(df)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked_C,Embarked_Q,Embarked_S
0,0,3,0,22.0,1,0,0,0,1
1,1,1,1,38.0,1,0,1,0,0
2,1,3,1,26.0,0,0,0,0,1
3,1,1,1,35.0,1,0,0,0,1
4,0,3,0,35.0,0,0,0,0,1


## 1.4 前処理:欠損値の補完

データには欠損値（不明な値：NaN）が存在する．  
NaNが入っている顧客データを利用するには,NaNを数値に変換する必要がある．  
処理の手法としては  
 - その顧客データの削除  
 - 平均値を代入  
 - 他の変数から重回帰  

などが挙げられる．  
以上の手法の中で一つを利用して欠損値を補完しなさい．  

In [12]:
df.shape

(889, 9)

In [13]:
df_2=df
df=df.dropna()
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age           0
SibSp         0
Parch         0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
dtype: int64

In [14]:
df.shape

(712, 9)

## 1.5 学習（決定木分析）
加工したデータを元にして学習を行い，生存者を推定する分類器を作成する．  

データを学習用のデータを分類器が正しく推定しているかどうかを検証する検証用データの２つに分割しなさい．  
比率は  
学習用：検証用　= 80：20  
とする．  
その後学習用データを利用し学習しなさい．  

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X=df.iloc[:,1:].values
y=df.iloc[:,0].values

In [17]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [18]:
from sklearn.tree import DecisionTreeClassifier

In [40]:
tree=DecisionTreeClassifier(criterion='entropy',max_depth=5,random_state=0)

In [41]:
tree.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=0, splitter='best')

In [42]:
y_pred=tree.predict(X_test)

## 1.6 評価  
学習した分類器が正しく推定できているかどうかを確かめる．  
検証用データを利用して，精度（生存者推定の正答率）を出力しなさい．  

In [43]:
from sklearn import metrics

In [44]:
print(metrics.accuracy_score(y_test,y_pred))

0.769230769231


## クロスバリデーションを行う

In [45]:
from sklearn.model_selection import cross_val_score

In [46]:
scores=cross_val_score(tree,X,y,cv=20,scoring='accuracy')

In [47]:
print(scores)

[ 0.72972973  0.83783784  0.81081081  0.81081081  0.75        0.83333333
  0.88888889  0.88888889  0.82857143  0.71428571  0.8         0.77142857
  0.77142857  0.8         0.74285714  0.8         0.85714286  0.85714286
  0.85714286  0.82857143]


In [48]:
scores.mean()

0.80894358644358655

## 1.7 前処理の比較  
一通り行った前処理を改めて見直し，精度を上げるために改善できそうな点を考慮して，再度  
前処理，学習，評価を行いなさい．  
その際に工夫した点を記述しなさい．  

Ageの列に欠損値が177個ありその行を全て消したので重要なデータも失ったかもしれない  
なので、欠損値を列の平均で補完してみる

In [49]:
#df_2

In [50]:
df_mean=df['Age'].mean()

In [51]:
df_2=df_2.fillna(df_mean)

In [52]:
df_2

,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked_C,Embarked_Q,Embarked_S
0,0,3,0,22.000000,1,0,0,0,1
1,1,1,1,38.000000,1,0,1,0,0
2,1,3,1,26.000000,0,0,0,0,1
3,1,1,1,35.000000,1,0,0,0,1
4,0,3,0,35.000000,0,0,0,0,1
5,0,3,0,29.642093,0,0,0,1,0
6,0,1,0,54.000000,0,0,0,0,1
7,0,3,0,2.000000,3,1,0,0,1
8,1,3,1,27.000000,0,2,0,0,1
9,1,2,1,14.000000,1,0,1,0,0


In [32]:
X_2=df_2.iloc[:,1:].values
y_2=df_2.iloc[:,0].values

In [53]:
tree_2=DecisionTreeClassifier(criterion='entropy',max_depth=5,random_state=0)

In [57]:
scores_2=cross_val_score(tree_2,X_2,y_2,cv=20,scoring='accuracy')

In [58]:
scores_2

array([ 0.75555556,  0.84444444,  0.77777778,  0.84444444,  0.73333333,
        0.77777778,  0.86666667,  0.91111111,  0.84444444,  0.84090909,
        0.81818182,  0.72727273,  0.75      ,  0.84090909,  0.81818182,
        0.75      ,  0.81818182,  0.88636364,  0.81818182,  0.88636364])

In [59]:
scores_2.mean()

0.81550505050505051

###### 結果  
あまり精度に変化がないように思える